In [ ]:
# load file a process to get the ctc matrix
SAMPLE = 2  # pick sample from the dev set

# match entity label to the ctc table
import os
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset
import soundfile as sf

CTC_DEPTH = 3  # size of the ctc matrix considered for search
NPATHS = 10 # number of longest paths on the bigram graph

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")


# path = "../data/dev/"
path = "../data/gtts/dev/"


file = str(SAMPLE) + '.wav'
speech, samplerate = sf.read(path+file)
i = int(file.split('.')[0]) - 1

input_values = tokenizer(speech, return_tensors="pt", padding="longest").input_values
logits = model(input_values).logits

# find where s_tokens appear in the table
ctc_table = torch.topk(logits, k=CTC_DEPTH, dim=-1)
predicted_ids = ctc_table.indices[0]
# predicted_ids = torch.argmax(logits, dim=-1).indices

# print(predicted_ids)
print(predicted_ids.shape)

# greedy decoding
transcription = tokenizer.batch_decode(torch.argmax(logits, dim=-1))[0].lower()
print(transcription)

# e: wiebke carolsfeld
# r: director

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# lexicon-constratined beam decode
import numpy as np

q = "what is a film directed by wiebke von carolsfeld"
lexicon = q.lower().split()
print(lexicon)

predictions = np.transpose(np.array(predicted_ids))

probs = torch.nn.functional.softmax(logits, dim = 1)
prediction_probs = np.transpose(probs.detach().numpy())

# find the position at which the word not in lexicon starts
w = ''
words = []
w_start_position = 0
for i, c_id in enumerate(predictions[0]):
    if c_id != 0:
        c = tokenizer.decode([c_id]).lower()
        if not w:
            w += c
        elif c != w[-1]:
            w += c
        if c_id == 4 and w:
            # check with lexicon
            print(w)
            if w in lexicon:
                words.append(w)
            else:
#                 print(w_start_position)
               
                break
#                 mismatch_position = i - len(w)
#                 print(mismatch_position)
#                 mismatch_character = tokenizer.decode([predictions[1][w_start_position]]).lower()
#                 print(mismatch_character)
            w = ''
            w_start_position = i
print(words)

# cross decode by picking all possible sequences from each string
v1 = predictions[0][w_start_position:]
v2 = predictions[1][w_start_position:]
v3 = predictions[2][w_start_position:]
print(v1)
print(v2)

In [ ]:
# all possible first letters
offset0 = 0
candidates = []
for i, c_id in enumerate(predictions[0][w_start_position:][offset0:]):
    if c_id not in [0,4]:
        c = tokenizer.decode([c_id]).lower()
        if c not in candidates:
            for w in lexicon:
                if w[:1] == c:
                    candidates.append(c)
                    break
        break
offset = i + 1

# alternative first letters
for i in range(CTC_DEPTH-1):
    for i, c_id in enumerate(predictions[i+1][w_start_position:][offset0:offset]):
        if c_id not in [0,4]:
            c = tokenizer.decode([c_id]).lower()
            if c not in candidates:
                for w in lexicon:
                    if w[:1] == c:
                        candidates.append(c)
                        break
print(candidates)

In [ ]:
# all possible second letters

def expand_sequences(predictions, candidates, offset0):
    new_candidates = []
    for i, c_id in enumerate(predictions[0][w_start_position:][offset0:]):
        if c_id not in [0,4]:
            c = tokenizer.decode([c_id]).lower()
            for prefix in candidates:
                prefix = prefix+c
                if prefix not in new_candidates:
                    for w in lexicon:
    #                     print(w[:len(prefix)], prefix)
                        if w[:len(prefix)] == prefix:
                            new_candidates.append(prefix)
                            break
            break
    offset = offset0 + i + 1

    # alternative first letters
    for i in range(CTC_DEPTH-1):
        for i, c_id in enumerate(predictions[i+1][w_start_position:][offset0:offset]):
            if c_id not in [0,4]:
                c = tokenizer.decode([c_id]).lower()
                for prefix in candidates:
                    prefix = prefix+c
                    if prefix not in new_candidates:
                        for w in lexicon:
                            if w[:len(prefix)] == prefix:
                                new_candidates.append(prefix)
                                break
    return new_candidates, offset


while candidates:
    print(candidates)
    candidates, offset = expand_sequences(predictions, candidates, offset)